# Getting started

This notebook provides an introduction to the tools that will be used for the _hands-on_ sessions of the MOOC "Path Integrals in Atomistic Modeling". Please follow the instructions to make sure that you have all the necessary components in place, and that you understand the basic mechanism to run simulations and analyze their outcome.  

## Installation

The notebooks associated with this course rely on some basic Python packages. If executing the following cell returns any errors, you should install the corresponding packages, e.g. using 

```
pip install -U numpy matplotlib ase chemiscope
```

`pimdmooc.py` is a small utility package that is present in the root folder of this repository.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ase, ase.io
import chemiscope
import pimdmooc
pimdmooc.add_ipi_paths()

ModuleNotFoundError: No module named 'chemiscope'

You should also have [i-PI](https://ipi-code.org) installed, and accessible from the path. This might require some more effort: if everything is configured correctly the following cell should return a prompt, and a message stating that 

```Simulation has already run for total_steps, will not even start. Modify total_steps or step counter to continue.```

In [ ]:
!i-pi 0-getting_started/do_nothing.xml

... and the following cell should generate a help string explaining the syntax for running `i-pi-driver`

In [ ]:
!i-pi-driver

If you get an error stating that the command has not been found, please open a terminal and follow these instructions.

_NB: this will install i-PI in your home folder, which will make these notebooks work out-of-the-box. if you know what you are doing, you can personalize the installation as long as `i-pi` and `i-pi-driver` are accessible from the path_

1. clone the i-PI repository

``` 
$ git clone https://github.com/i-pi/i-pi.git 
```

2. add the i-pi folder to the default path

```
$ echo ". ~/i-pi/env.sh" >> .bashrc
```

3. compile the driver files 

```
$ cd i-pi/drivers/f90/ & make
```

4. try to execute again the cells above

## Running i-PI

i-PI functions following a client-server protocol, in which i-PI acts as the server, executing advanced MD simulations, while the evaluation of energies and forces is delegated to an external code, that acts as the client

![a scheme of the i-PI client-server model](figures/ipi-scheme.png)

Thus, to run i-PI you need to launch (at least) two processes: `i-pi` and a driver -- here we use a minimalistic FORTRAN tool that can compute energy and forces according to a number of simple potential energy models. 

1. open *two* terminals
2. in the first terminal launch `i-pi`

```
$ cd pimd-mooc/0-getting_started
$ i-pi input.xml
```

3. in the second terminal, launch the driver

```
$ i-pi-driver -u -h driver -m pswater -v
```

_NB: `i-pi-driver` does not need input files nor generate outputs, so you can run it from any folder_

Both programs run in verbose mode, so you can see the communication logs between the two. 

```
 @SOCKET:   Client asked for connection from . Now hand-shaking.
 @SOCKET:   Handshaking was successful. Added to the client list.
 @SOCKET: 21/09/17-15:49:35 Assigning [ none] request id    0 to client with last-id None (  0/  1 : )
 @SOCKET: 21/09/17-15:49:35 Assigning [match] request id    0 to client with last-id    0 (  0/  1 : )
 # Average timings at MD step       0. t/step: 4.08595e-02
 @SOCKET: 21/09/17-15:49:35 Assigning [match] request id    0 to client with last-id    0 (  0/  1 : )
 # Average timings at MD step       1. t/step: 3.96178e-02
 @SOCKET: 21/09/17-15:49:35 Assigning [match] request id    0 to client with last-id    0 (  0/  1 : )
```

and 

```
  Message from server: STATUS
  Message from server: POSDATA
  Message from server: STATUS
  Message from server: GETFORCE
  Message from server: STATUS
  Message from server: STATUS
  Message from server: POSDATA
  Message from server: STATUS
  Message from server: GETFORCE
  Message from server: STATUS
```

You can try to kill the driver with `CTRL+C` and see what happens. `i-pi` should stop and wait for the connection of another client: if you launch `i-pi-driver` again, the simulation will continue. This robust management of multiple client allows the implementation of a trivial level of parallelism when the advanced MD simulation require the calculation of multiple replicas, as we will see in the following exercises.

_NB: if `i-pi` exits abruptly (e.g. by closing down a shell) it will leave a UNIX domain socket file in `/tmp/`, named `/tmp/ipi-NAME`. If you launch again i-PI it will exit with an error message similar to_

```
Error opening unix socket. Check if a file /tmp/ipi_driver exists, and remove it if unused.
```

_needless to say: if you are reasonably confident this has been left around by a previous run, follow the instructions and remove the file._

### i-PI input file format

i-PI uses XML-formatted input files, that describe how the simulation is set up, where to get energy and forces from, and how to output the results of the simulation. in this course we will mostly use prepared input files, where the meaning of the parameters for each specific application will be explained only in relation to the concepts being covered. If you want to learn more about i-PI, you can visit the [website](https://ipi-code.org) or read the [documentation](https://ipi-code.org/i-pi/).

In [ ]:
!cat 0-getting_started/input.xml   # the meaning of most of these options will become clear as the course progresses

## Analyzing the results

After you have run your simulations, you can look into the output files, load them and visualize them straight from the notebooks. i-PI does not have a pre-defined output format, and each run can be configured to output multiple files with different content and strides. 

The `<properties>` outputs contain properties of the system as a whole, such as the timestamp, temperature or potential energy of the system. The header of the file contains a summary of its content. In this simple example, the output is printed to `simulation.out`. _NB: Unless explicitly specified, *all* quantities read and output by i-PI are in Hartree atomic units_

In [ ]:
!head 0-getting_started/simulation.out

In [ ]:
simulation_data = np.loadtxt("0-getting_started/simulation.out")

In [ ]:
plt.plot(simulation_data[:,0], simulation_data[:,1])
plt.xlabel("time / ps"); plt.ylabel("temperature / K")

Atomic positions and properties are dumped to files based on the settings given in a `<trajectory>` tag. 
Here we use a combined visualizer called `chemiscope`, but obviously you can use alternative tools for this purpose.

In [ ]:
trajectory_data = pimdmooc.read_ipi_xyz("0-getting_started/simulation.pos_0.xyz")

`chemiscope` allows you to visualize simultaneously properties and configurations

In [ ]:
chemiscope.show(frames=trajectory_data, 
                properties=dict(
                    time=simulation_data[::10,0], 
                    temperature=simulation_data[::10,1],
                    potential=simulation_data[::10,3]
                               )
               )

given that sometimes it is not trivial to set up a jupyter widget to load properly, if you have problems opening the visualization above you can also export the visualization as a .json file, that can be loaded on [chemiscope.org](https://chemiscope.org)

In [ ]:
chemiscope.write_input("0-example.json.gz",
                frames=trajectory_data, 
                properties=dict(
                    time=simulation_data[::10,0], 
                    temperature=simulation_data[::10,1],
                    potential=simulation_data[::10,3]
                               )
               )

In [4]:
import ase

ModuleNotFoundError: No module named 'ase'

In [5]:
import chemiscope

In [7]:
import ase.io

In [13]:
frames = ase.io.read("0-getting_started/simulation.pos_0.xyz", ":")

In [16]:
chemiscope.show(frames)

ChemiscopeWidget(value=None, data='{"meta": {"name": " "}, "structures": [{"size": 3, "names": ["O", "H", "H"]…

In [12]:
frames

[]